# AutoGEN 第二课： 人工如何部分参与agent的决策过程以及死循环情况的处理

通过第一个例子，我们初步了解了在没有人工干预的情况下，Autogen框架是如何控制2个不同种类的agent通过对话完成一个任务的。
但是在绝大多数的情况下，我们都希望人工能够参与进去，并且在合适的时候给出纠正或者提出其他的需求。
下面这个例子中，我们逐步递进，介绍人工如何部分参与决策的过程。
这里的部分参与你可以理解为：在正常情况下agent都拥有足够的智能，可以处理大部份的情况。只有当陷入了难以决策的困难时，才会寻求人类的帮助。

在代码中表现为，assistant agent会在合适的时候返回"TERMINATE"暗号，跟user agent说了声，“这样搞不是个事啊，先停一停，听听老板啥意见”， 这个时候user agent就会停下来等待人类的指令。 在得到新指令的情况下，并将新指令传达给assistant agent, 继续干活，否则流程就会在这个地方一直停下来。 

当然，除了暗号的形式外，还有一个常见的做法就是设定对话的最后轮次数量，当2个或多个agent对话的轮次达到最大限制的时候，也会停下来等人类的指定输入了。

## 1. 配置环境

1. 你需要安装autogent的库， 要求python版本>=3.8
2. 配置openai 或者 azure openai的api key, 并将key写入到一个名为 "OAI_CONFIG_LIST"的文件中，并将该文件放在项目目录下。或者将其配置到你的环境变量中。

OAI_CONFIG_LIST的内容结构如下：
```python
[
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },
    {
        'model': 'gpt-4',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
    {
        'model': 'gpt-4-32k',
        'api_key': '<your Azure OpenAI API key here>',
        'api_base': '<your Azure OpenAI API base here>',
        'api_type': 'azure',
        'api_version': '2023-06-01-preview',
    },
]

In [3]:
%pip install pyautogen~=0.1.0 -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4", "gpt-4-0314", "gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    },
)

# 构建一个assistant agent实例，并且名为 assistant
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config={
        "seed": 42,  # 用于复现结果的种子
        "config_list": config_list,  
        "temperature": 0,  
    },  
)

# 构建一个user proxy agent, 并命名为user_proxy
user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    #  不同的人类介入程度：
    #   never: 人类始终不介入， 
    #   always: 每一步都需要人类介入， 
    #   terminate：只有当assistant agent认为当前交互结果没问题了，会返回一个"terminate"字符串, （你可以理解为暗号）。此时user proxy agent 才会停下等人类的下一步指令。
    human_input_mode="TERMINATE",  
    max_consecutive_auto_reply=10,      # 最大的交互轮数，如果超过了会强制退出，等人类指令。
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"), # 如果别的agent返回的结果中带有这个"TERMINATE"标记，也会强制退出，等人类指令。
    code_execution_config={
        "work_dir": "coding",   # 临时代码存放的目录，为coding
        "use_docker": False,    # 设定是否使用docker
    },
    # 给当前的agent设定独有的prompt
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet."""
)

# 启动两个agent会话
user_proxy.initiate_chat(
    assistant,
    message="""Who should read this paper: https://arxiv.org/abs/2308.08155""",
)

user_proxy (to assistant):

Who should read this paper: https://arxiv.org/abs/2308.08155

--------------------------------------------------------------------------------
assistant (to user_proxy):

To determine who should read the paper, we need to first understand the content and context of the paper. We can do this by fetching the abstract of the paper from the provided URL and analyzing it. 

Here is a Python script that uses the BeautifulSoup library to scrape the abstract from the webpage. 

```python
# filename: fetch_abstract.py

import requests
from bs4 import BeautifulSoup

def fetch_abstract(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    abstract = soup.find('blockquote', attrs={'class': 'abstract mathjax'}).text.strip()
    return abstract

url = "https://arxiv.org/abs/2308.08155"
print(fetch_abstract(url))
```

Please save this script as `fetch_abstract.py` and run it. This will print the abstract of the paper. Based on th